In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)
 
os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns

import pickle 
import logging


from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/re_decisioning
/Users/shashankgupta/Documents/code/git_project/re_decisioning/Code


In [2]:
import sys
sys.path.insert(0, '/Users/sarfrazahmed/Documents/projects/deposit-model/conf')
import pandas as pd
import numpy as np
import chart_studio.plotly as py 
from datetime import timedelta
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio
from pandas_profiling import ProfileReport
pio.templates.default = 'plotly_dark'
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
color_map = {'high': '#ef553b', 'low': '#636efa'}
import datetime
import pytz

In [3]:
querySno = SQLQuery('snowflake')
rev_df = querySno(r"""
with lending_data as (
select business_id, due_date 
FROM (select distinct business_id, due_date from PROD_DB.ADHOC.LENDING_DPD_DATA_PILOT_GA) pay

-- and pay.lending_business_id = 'cf6d8b20-574f-4362-b1f5-274bb2231c05'
order by due_date
)

,txns_with_categories AS (
select a.business_id, d.due_date,DATEDIFF(month, TRANSACTION_DATE, d.due_date) AS txn_month, amount, description,
			REGEXP_REPLACE(REGEXP_REPLACE(lower(description), ':',' '), '[^a-z ]', '') AS description_processed, transaction_date,
			            CASE WHEN description_processed ILIKE any(
                                                '%%novo funding%%',
			            						'%% earninactivehours %%', 
                                                'earninactivehours %%',
                                                '%% earninactivehours',
                                                'earninactivehours',
			            						'%% activehours %%', 
                                                'activehours %%', 
                                                '%% activehours', 
                                                'activehours', 
			            						'%% navchek %%', 
                                                'navchek %%', 
                                                '%% navchek', 
                                                'navchek', 
			            						'%% loan %%', 
                                                'loan %%',
                                                '%% loan',
                                                'loan',
			            						'%% earnin %%',
                                                'earnin %%',
                                                '%% earnin',
                                                'earnin', 
			            						'%% advance %%',
                                                '%% advance',
                                                'advance %%',
                                                'advance'
			            ) THEN 1 ELSE 0 END AS LOANS,
			            CASE WHEN description_processed ILIKE any(
			            						'%% hcclaimpmt %%',
                                                '%% hcclaimpmt',
                                                'hcclaimpmt %%',
                                                'hcclaimpmt',
												 '%% unitedhealthcare %%',
                                                 'unitedhealthcare %%',
                                                 '%% unitedhealthcare',
                                                 'unitedhealthcare',
												 '%% bcbs %%',
                                                 '%% bcbs',
                                                 'bcbs %%',
                                                 'bcbs',
												 '%% cigna %%',
                                                 'cigna %%',
                                                 '%% cigna',
                                                 'cigna',
												 '%% govt %%',
                                                 '%% govt',
                                                 'govt %%',
                                                 'govt',
												 '%% humana %%',
                                                 '%% humana',
                                                 'humana %%',
                                                 'humana',
												 '%% aetna %%',
                                                 'aetna %%',
                                                 '%% aetna',
                                                 'aetna',
												 '%% insurance %%',
                                                 'insurance %%',
                                                 '%% insurance',
                                                 'insurance',
												 '%% life %%',
                                                 'life %%',
                                                 '%% life',
                                                 'life',
												 '%% blue %%',
                                                 '%% blue',
                                                 'blue %%',
                                                 'blue'
						) THEN 1 ELSE 0 END AS INSURANCE,
						CASE WHEN description_processed ILIKE any(
												'%% sbtpg %%',
                                                '%% sbtpg',
                                                'sbtpg %%',
                                                'sbtpg', 
												'%% xxtaxeip %%',
                                                'xxtaxeip %%',
                                                '%% xxtaxeip',
                                                'xxtaxeip', 
												'%% tax %%', 
                                                'tax %%', 
                                                '%% tax', 
                                                'tax', 
												'%% irs %%', 
                                                '%% irs', 
                                                'irs %%', 
                                                'irs', 
												'%% treas %%', 
                                                '%% treas', 
                                                'treas %%', 
                                                'treas', 
												'%% taxrfd %%',
                                                '%% taxrfd',
                                                'taxrfd %%',
                                                'taxrfd'
						) THEN 1 ELSE 0 END AS TAX
from fivetran_db.PROD_NOVO_API_PUBLIC.transactions a inner join lending_data d on a.business_id = d.business_id
		   WHERE datediff(MONTH ,TRANSACTION_DATE,d.due_date) < 2 and datediff(MONTH ,TRANSACTION_DATE,d.due_date) >= 0
           AND status='active' AND TYPE = 'credit'     
)
,all_credits AS (
SELECT a.BUSINESS_ID, due_date,txn_month,
               sum(amount)  AS total_credit_amount, 
               sum(CASE WHEN LOANS = 1 THEN amount ELSE 0 end) AS total_loan_credit_amount, 
               sum(CASE WHEN INSURANCE = 1 THEN amount ELSE 0 END) AS total_insurance_credit_amount, 
               sum(CASE WHEN TAX = 1 THEN amount ELSE 0 end) AS total_tax_credit_amount
FROM  txns_with_categories a
GROUP BY 1,2,3
),
all_txn_credits AS (
SELECT business_id, due_date,txn_month, 
               COALESCE(total_credit_amount, 0) AS total_credit_amount,
               COALESCE(total_loan_credit_amount, 0) AS total_loan_credit_amount,
               COALESCE(total_insurance_credit_amount, 0) AS total_insurance_credit_amount,
               COALESCE(total_tax_credit_amount, 0) AS total_tax_credit_amount
            --   COALESCE(TOTAL_CREDIT_AMOUNT_PLAID, 0) AS TOTAL_CREDIT_AMOUNT_PLAID,
            --   COALESCE(TOTAL_LOAN_CREDIT_AMT_PLAID, 0) AS TOTAL_LOAN_CREDIT_AMT_PLAID,
            --   COALESCE(TOTAL_INSURANCE_CREDIT_AMT_PLAID, 0) AS TOTAL_INSURANCE_CREDIT_AMT_PLAID,
            --   COALESCE(TOTAL_TAX_CREDIT_AMT_PLAID, 0) AS TOTAL_TAX_CREDIT_AMT_PLAID
FROM all_credits 
)
,last_6mnth_revenue as (
SELECT business_id, due_date,
       CASE WHEN txn_month = 0 THEN 'past_1month_rev'
            WHEN txn_month = 1 THEN 'past_2month_rev'
            WHEN txn_month = 2 THEN 'past_3month_rev'
            WHEN txn_month in (4,5,6) THEN 'past_furthest_3months'
       END as month,
       ((total_credit_amount-total_loan_credit_amount-total_insurance_credit_amount-total_tax_credit_amount)
	           ) AS revenue
FROM all_txn_credits
where txn_month <= 1)

SELECT business_id, due_date, revenue_past_1m, revenue_past_2m, revenue_past_1m + revenue_past_2m as total_revenue_2m, revenue_past_1m/(1 + revenue_past_2m) as ratio_revenue_1m_2m from (
select business_id,due_date, revenue as revenue_past_1m, lead(revenue) over(order by business_id, due_date, month) as revenue_past_2m,
row_number() over(partition by business_id, due_date order by month) rnk
from (
select business_id, due_date, month, sum(revenue) revenue
from last_6mnth_revenue
-- where business_id = '0d39a85e-12dc-4487-904a-55cc24c397bf'
group by 1,2,3
order by 1,2,3
)
)
where rnk = 1
"""
)

In [4]:
rev_df.head()

,business_id,due_date,revenue_past_1m,revenue_past_2m,total_revenue_2m,ratio_revenue_1m_2m
0,84d80cf9-adc2-4e04-9248-0c6dc5a398b7,2022-11-02,654.33,529.41,1183.74,1.233631
1,efe8f866-4a39-4550-82c9-77dcec91cdd1,2023-03-05,14823.67,6995.16,21818.83,2.118829
2,6c45c434-a32a-499e-9155-3d48c1724649,2023-03-13,11491.66,11494.97,22986.63,0.999625
3,6c5c8be7-f46b-4f7a-98d0-cb777add49fd,2022-10-27,1184.09,1091.15,2275.24,1.084183
4,24d94434-ca7c-4e2c-a8d9-ea03d348c30a,2022-09-21,2695.10,2370.79,5065.89,1.136315


In [5]:
rev_df.shape

(4967, 6)

In [6]:
txn_df = querySno(r"""
select business_id, due_date, 
avg(count_pd_deposit_1m)count_pd_deposit_1m, avg(count_ach_debit_1m)count_ach_debit_1m, avg(count_ach_credit_1m)count_ach_credit_1m, avg(count_card_withdrawal_1m)count_card_withdrawal_1m, avg(count_mrdc_credit_1m)count_mrdc_credit_1m,
avg(count_pd_deposit_2m)count_pd_deposit_2m, avg(count_ach_debit_2m)count_ach_debit_2m, avg(count_ach_credit_2m)count_ach_credit_2m, avg(count_card_withdrawal_2m)count_card_withdrawal_2m, avg(count_mrdc_credit_2m)count_mrdc_credit_2m,
avg(amount_pd_deposit_1m)amount_pd_deposit_1m, avg(amount_ach_debit_1m)amount_ach_debit_1m, avg(amount_ach_credit_1m)amount_ach_credit_1m, avg(amount_card_withdrawal_1m)amount_card_withdrawal_1m, avg(amount_mrdc_credit_1m)amount_mrdc_credit_1m, 
avg(amount_pd_deposit_2m)amount_pd_deposit_2m, avg(amount_ach_debit_2m)amount_ach_debit_2m, avg(amount_ach_credit_2m)amount_ach_credit_2m, avg(amount_card_withdrawal_2m)amount_card_withdrawal_2m, avg(amount_mrdc_credit_2m)amount_mrdc_credit_2m,
avg(pd_deposit_size_1m)pd_deposit_size_1m, avg(ach_debit_size_1m)ach_debit_size_1m, avg(ach_credit_size_1m)ach_credit_size_1m, avg(card_withdrawal_size_1m)card_withdrawal_size_1m, avg(mrdc_credit_size_1m)mrdc_credit_size_1m, 
avg(pd_deposit_size_2m)pd_deposit_size_2m, avg(ach_debit_size_2m)ach_debit_size_2m, avg(ach_credit_size_2m)ach_credit_size_2m, avg(card_withdrawal_size_2m)card_withdrawal_size_2m, avg(mrdc_credit_size_2m)mrdc_credit_size_2m,
avg(amount_credited_1m)amount_credited_1m, avg(distinct_credit_txns_1m)distinct_credit_txns_1m, avg(credit_size_1m)credit_size_1m, avg(median_amount_credited_1m)median_amount_credited_1m, avg(stddev_amount_credited_1m)stddev_amount_credited_1m, 
avg(amount_credited_2m)amount_credited_2m, avg(distinct_credit_txns_2m)distinct_credit_txns_2m, avg(credit_size_2m)credit_size_2m, avg(median_amount_credited_2m)median_amount_credited_2m, avg(stddev_amount_credited_2m)stddev_amount_credited_2m,
avg(amount_debited_1m)amount_debited_1m, avg(distinct_debit_txns_1m)distinct_debit_txns_1m, avg(debit_size_1m)debit_size_1m, avg(median_amount_debited_1m)median_amount_debited_1m, avg(stddev_amount_debited_1m)stddev_amount_debited_1m, 
avg(amount_debited_2m)amount_debited_2m, avg(distinct_debit_txns_2m)distinct_debit_txns_2m, avg(debit_size_2m)debit_size_2m, avg(median_amount_debited_2m)median_amount_debited_2m, avg(stddev_amount_debited_2m)stddev_amount_debited_2m,
avg(ratio_debit_credit_1m)ratio_debit_credit_1m, avg(ratio_debit_credit_frequency_1m)ratio_debit_credit_frequency_1m, avg(ratio_debit_credit_2m)ratio_debit_credit_2m, avg(ratio_debit_credit_frequency_2m)ratio_debit_credit_frequency_2m,
avg(ratio_pd_amt_total_credit_1m)ratio_pd_amt_total_credit_1m, avg(ratio_ach_debit_total_debit_1m)ratio_ach_debit_total_debit_1m, avg(ratio_ach_credit_total_credit_1m)ratio_ach_credit_total_credit_1m, avg(ratio_card_withdrawal_total_debit_1m)ratio_card_withdrawal_total_debit_1m, avg(ratio_mrdc_credit_total_credit_1m)ratio_mrdc_credit_total_credit_1m, 
avg(ratio_pd_amt_total_credit_2m)ratio_pd_amt_total_credit_2m, avg(ratio_ach_debit_total_debit_2m)ratio_ach_debit_total_debit_2m, avg(ratio_ach_credit_total_credit_2m)ratio_ach_credit_total_credit_2m, avg(ratio_card_withdrawal_total_debit_2m)ratio_card_withdrawal_total_debit_2m, avg(ratio_mrdc_credit_total_credit_2m)ratio_mrdc_credit_total_credit_2m,
avg(ratio_pd_freq_total_credit_1m)ratio_pd_freq_total_credit_1m, avg(ratio_ach_debit_freq_total_debit_1m)ratio_ach_debit_freq_total_debit_1m, avg(ratio_ach_credit_freq_total_credit_1m)ratio_ach_credit_freq_total_credit_1m, avg(ratio_card_withdrawal_freq_total_debit_1m)ratio_card_withdrawal_freq_total_debit_1m, avg(ratio_mrdc_credit_freq_total_credit_1m)ratio_mrdc_credit_freq_total_credit_1m, 
avg(ratio_pd_freq_total_credit_2m)ratio_pd_freq_total_credit_2m, avg(ratio_ach_debit_freq_total_debit_2m)ratio_ach_debit_freq_total_debit_2m, avg(ratio_ach_credit_freq_total_credit_2m)ratio_ach_credit_freq_total_credit_2m, avg(ratio_card_withdrawal_freq_total_debit_2m)ratio_card_withdrawal_freq_total_debit_2m, avg(ratio_mrdc_credit_freq_total_credit_2m)ratio_mrdc_credit_freq_total_credit_2m,
avg(ratio_ach_debit_credit_1m)ratio_ach_debit_credit_1m, avg(ratio_ach_debit_credit_2m)ratio_ach_debit_credit_2m,
avg(ratio_ach_debit_credit_freq_1m)ratio_ach_debit_credit_freq_1m, avg(ratio_ach_debit_credit_freq_2m)ratio_ach_debit_credit_freq_2m,
avg(ach_d_size_1m)ach_d_size_1m, avg(ach_d_size_2m)ach_d_size_2m, avg(ach_c_size_1m)ach_c_size_1m, avg(ach_c_size_2m)ach_c_size_2m


from (
select pay.business_id, pay.due_date, t.transaction_date, t.amount, t.type, t.running_balance, pay.min_created_at,
datediff(day, t.transaction_date, pay.due_date) as DaysFromDueDate, datediff(day, pay.min_created_at, t.transaction_date) as DaysSinceLoanTaken,
CASE WHEN TYPE = 'credit' and DESCRIPTION  ILIKE ANY(
	                    '%%AMAZON%%'
	                    , '%%AMZN%%'
	                    , '%%STRIPE%%'
	                    , '%%SQUARE INC%%'
	                    , '%%SHOPIFY%%'
	                    , '%%SHOPPAY%%'
	                    )
                    AND NOT(
	                    description ILIKE ANY(
	                        '%%LYFT'
	                        , '%%OfferUp%%'
	                        , '%%Gumroad%%'
	                        , '%%FB%%Fundrai%%'
	                        , '%%Verify%%'
	                        , '%%CASH%%'
	                        , '%%PAYROLL%%'
	                        , '%%VRFY%%'
	                        , '%%CAPITAL%%'
	                        , '%%REFUND%%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
CASE WHEN TYPE ='debit' AND MEDIUM  ILIKE ANY('%%external withdrawal%%', '%%iat withdrawal%%') THEN 1 ELSE 0 END AS ach_debit_txn,
CASE WHEN type = 'credit'  AND medium ILIKE ANY('%%external deposit%%', '%%iat deposit%%') THEN 1 ELSE 0 END AS ach_credit_txn,
CASE WHEN medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
CASE WHEN TYPE = 'credit' AND description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,

sum(case when daysFromDueDate <= 30 then PD_DEPOSIT_TXN else 0 end) over (partition by biz.id, due_date) as count_pd_deposit_1m,
sum(case when daysFromDueDate <= 30 then ach_debit_txn else 0 end) over (partition by biz.id, due_date) as count_ach_debit_1m,
sum(case when daysFromDueDate <= 30 then ach_credit_txn else 0 end) over (partition by biz.id, due_date) as count_ach_credit_1m,
sum(case when daysFromDueDate <= 30 then card_withdrawal_txn else 0 end) over (partition by biz.id, due_date) as count_card_withdrawal_1m,
sum(case when daysFromDueDate <= 30 then mrdc_credit_txn else 0 end) over (partition by biz.id, due_date) as count_mrdc_credit_1m,

sum(case when daysFromDueDate <= 60 then PD_DEPOSIT_TXN else 0 end) over (partition by biz.id, due_date) as count_pd_deposit_2m,
sum(case when daysFromDueDate <= 60 then ach_debit_txn else 0 end) over (partition by biz.id, due_date) as count_ach_debit_2m,
sum(case when daysFromDueDate <= 60 then ach_credit_txn else 0 end) over (partition by biz.id, due_date) as count_ach_credit_2m,
sum(case when daysFromDueDate <= 60 then card_withdrawal_txn else 0 end) over (partition by biz.id, due_date) as count_card_withdrawal_2m,
sum(case when daysFromDueDate <= 60 then mrdc_credit_txn else 0 end) over (partition by biz.id, due_date) as count_mrdc_credit_2m,

sum(case when daysFromDueDate <= 90 then PD_DEPOSIT_TXN else 0 end) over (partition by biz.id, due_date) as count_pd_deposit_3m,
sum(case when daysFromDueDate <= 90 then ach_debit_txn else 0 end) over (partition by biz.id, due_date) as count_ach_debit_3m,
sum(case when daysFromDueDate <= 90 then ach_credit_txn else 0 end) over (partition by biz.id, due_date) as count_ach_credit_3m,
sum(case when daysFromDueDate <= 90 then card_withdrawal_txn else 0 end) over (partition by biz.id, due_date) as count_card_withdrawal_3m,
sum(case when daysFromDueDate <= 90 then mrdc_credit_txn else 0 end) over (partition by biz.id, due_date) as count_mrdc_credit_3m,

sum(case when daysFromDueDate <= 30 and PD_DEPOSIT_TXN = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_pd_deposit_1m,
sum(case when daysFromDueDate <= 30 and ach_debit_txn = 1 then -amount else 0 end) over (partition by biz.id, due_date) as amount_ach_debit_1m,
sum(case when daysFromDueDate <= 30 and ach_credit_txn = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_ach_credit_1m,
sum(case when daysFromDueDate <= 30 and card_withdrawal_txn = 1 then -amount else 0 end) over (partition by biz.id, due_date) as amount_card_withdrawal_1m,
sum(case when daysFromDueDate <= 30 and mrdc_credit_txn = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_mrdc_credit_1m,

sum(case when daysFromDueDate <= 60 and PD_DEPOSIT_TXN = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_pd_deposit_2m,
sum(case when daysFromDueDate <= 60 and ach_debit_txn = 1 then -amount else 0 end) over (partition by biz.id, due_date) as amount_ach_debit_2m,
sum(case when daysFromDueDate <= 60 and ach_credit_txn = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_ach_credit_2m,
sum(case when daysFromDueDate <= 60 and card_withdrawal_txn = 1 then -amount else 0 end) over (partition by biz.id, due_date) as amount_card_withdrawal_2m,
sum(case when daysFromDueDate <= 60 and mrdc_credit_txn = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_mrdc_credit_2m,

sum(case when daysFromDueDate <= 90 and PD_DEPOSIT_TXN = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_pd_deposit_3m,
sum(case when daysFromDueDate <= 90 and ach_debit_txn = 1 then -amount else 0 end) over (partition by biz.id, due_date) as amount_ach_debit_3m,
sum(case when daysFromDueDate <= 90 and ach_credit_txn = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_ach_credit_3m,
sum(case when daysFromDueDate <= 90 and card_withdrawal_txn = 1 then -amount else 0 end) over (partition by biz.id, due_date) as amount_card_withdrawal_3m,
sum(case when daysFromDueDate <= 90 and mrdc_credit_txn = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_mrdc_credit_3m,

amount_pd_deposit_1m/(1 + count_pd_deposit_1m) as pd_deposit_size_1m, amount_pd_deposit_2m/(1 + count_pd_deposit_2m) as pd_deposit_size_2m, amount_pd_deposit_3m/(1 + count_pd_deposit_3m) as pd_deposit_size_3m,
amount_ach_debit_1m/(count_ach_debit_1m + 1) as ach_debit_size_1m, amount_ach_debit_2m/(count_ach_debit_2m + 1) as ach_debit_size_2m, amount_ach_debit_3m/(count_ach_debit_3m + 1) as ach_debit_size_3m,
amount_ach_credit_1m/(count_ach_credit_1m + 1) as ach_credit_size_1m, amount_ach_credit_2m/(count_ach_credit_2m + 1) as ach_credit_size_2m, amount_ach_credit_3m/(count_ach_credit_3m + 1) as ach_credit_size_3m,
amount_card_withdrawal_1m/(count_card_withdrawal_1m + 1) as card_withdrawal_size_1m, amount_card_withdrawal_2m/(count_card_withdrawal_2m + 1) as card_withdrawal_size_2m, amount_card_withdrawal_3m/(count_card_withdrawal_3m + 1) as card_withdrawal_size_3m,
amount_mrdc_credit_1m/(count_mrdc_credit_1m + 1) as mrdc_credit_size_1m, amount_mrdc_credit_2m/(count_mrdc_credit_2m + 1) as mrdc_credit_size_2m, amount_mrdc_credit_3m/(count_mrdc_credit_3m + 1) as mrdc_credit_size_3m,

sum(case when running_balance < 0 and daysFromDueDate <= 30 then 1 else 0 end) over (partition by biz.id, due_date) as od_count_1m,
sum(case when running_balance = 0 and daysFromDueDate <= 30 then 1 else 0 end) over (partition by biz.id, due_date) as zero_balance_count_1m,
avg(case when daysFromDueDate <= 30 then running_balance else 0 end) over (partition by biz.id, due_date) as avg_running_balance_1m,
median(case when daysFromDueDate <= 30 then running_balance else 0 end) over (partition by biz.id, due_date) as median_running_balance_1m,
stddev(case when daysFromDueDate <= 30 then running_balance else 0 end) over (partition by biz.id, due_date) as stddev_running_balance_1m,

sum(case when running_balance < 0 and daysFromDueDate <= 60 then 1 else 0 end) over (partition by biz.id, due_date) as od_count_2m,
sum(case when running_balance = 0 and daysFromDueDate <= 60 then 1 else 0 end) over (partition by biz.id, due_date) as zero_balance_count_2m,
avg(case when daysFromDueDate <= 60 then running_balance else 0 end) over (partition by biz.id, due_date) as avg_running_balance_2m,
median(case when daysFromDueDate <= 60 then running_balance else 0 end) over (partition by biz.id, due_date) as median_running_balance_2m,
stddev(case when daysFromDueDate <= 60 then running_balance else 0 end) over (partition by biz.id, due_date) as stddev_running_balance_2m,

sum(case when running_balance < 0 and daysFromDueDate <= 90 then 1 else 0 end) over (partition by biz.id, due_date) as od_count_3m,
sum(case when running_balance = 0 and daysFromDueDate <= 90 then 1 else 0 end) over (partition by biz.id, due_date) as zero_balance_count_3m,
avg(case when daysFromDueDate <= 90 then running_balance else 0 end) over (partition by biz.id, due_date) as avg_running_balance_3m,
median(case when daysFromDueDate <= 90 then running_balance else 0 end) over (partition by biz.id, due_date) as median_running_balance_3m,
stddev(case when daysFromDueDate <= 90 then running_balance else 0 end) over (partition by biz.id, due_date) as stddev_running_balance_3m,

sum(case when daysFromDueDate <= 30 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as amount_credited_1m,
sum(case when daysFromDueDate <= 30 and type = 'credit' then 1 else 0 end) over (partition by biz.id, due_date) as distinct_credit_txns_1m,
amount_credited_1m/(distinct_credit_txns_1m + 1) as credit_size_1m,
median(case when daysFromDueDate <= 30 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as median_amount_credited_1m,
stddev(case when daysFromDueDate <= 30 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as stddev_amount_credited_1m,

sum(case when daysFromDueDate <= 30 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as amount_debited_1m,
sum(case when daysFromDueDate <= 30 and type = 'debit' then 1 else 0 end) over (partition by biz.id, due_date) as distinct_debit_txns_1m,
amount_debited_1m/(distinct_debit_txns_1m + 1) as debit_size_1m,
median(case when daysFromDueDate <= 30 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as median_amount_debited_1m,
stddev(case when daysFromDueDate <= 30 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as stddev_amount_debited_1m,

amount_debited_1m/(amount_credited_1m + 1) as ratio_debit_credit_1m,
distinct_debit_txns_1m/(distinct_credit_txns_1m + 1) as ratio_debit_credit_frequency_1m,

sum(case when daysFromDueDate <= 60 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as amount_credited_2m,
sum(case when daysFromDueDate <= 60 and type = 'credit' then 1 else 0 end) over (partition by biz.id, due_date) as distinct_credit_txns_2m,
amount_credited_2m/(distinct_credit_txns_2m + 1) as credit_size_2m,
median(case when daysFromDueDate <= 60 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as median_amount_credited_2m,
stddev(case when daysFromDueDate <= 60 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as stddev_amount_credited_2m,

sum(case when daysFromDueDate <= 60 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as amount_debited_2m,
sum(case when daysFromDueDate <= 60 and type = 'debit' then 1 else 0 end) over (partition by biz.id, due_date) as distinct_debit_txns_2m,
amount_debited_2m/(distinct_debit_txns_2m + 1) as debit_size_2m,
median(case when daysFromDueDate <= 60 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as median_amount_debited_2m,
stddev(case when daysFromDueDate <= 60 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as stddev_amount_debited_2m,

amount_debited_2m/(amount_credited_2m + 1) as ratio_debit_credit_2m,
distinct_debit_txns_2m/(distinct_credit_txns_2m + 1) as ratio_debit_credit_frequency_2m,

sum(case when daysFromDueDate <= 90 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as amount_credited_3m,
sum(case when daysFromDueDate <= 90 and type = 'credit' then 1 else 0 end) over (partition by biz.id, due_date) as distinct_credit_txns_3m,
amount_credited_3m/(distinct_credit_txns_3m + 1) as credit_size_3m,
median(case when daysFromDueDate <= 90 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as median_amount_credited_3m,
stddev(case when daysFromDueDate <= 90 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as stddev_amount_credited_3m,

sum(case when daysFromDueDate <= 90 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as amount_debited_3m,
sum(case when daysFromDueDate <= 90 and type = 'debit' then 1 else 0 end) over (partition by biz.id, due_date) as distinct_debit_txns_3m,
amount_debited_3m/(distinct_debit_txns_3m + 1) as debit_size_3m,
median(case when daysFromDueDate <= 90 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as median_amount_debited_3m,
stddev(case when daysFromDueDate <= 90 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as stddev_amount_debited_3m,

amount_debited_3m/(amount_credited_3m + 1) as ratio_debit_credit_3m,
distinct_debit_txns_3m/(distinct_credit_txns_3m + 1) as ratio_debit_credit_frequency_3m,

COALESCE(amount_pd_deposit_1m/NULLIF(amount_credited_1m, 0), 0) as ratio_pd_amt_total_credit_1m, COALESCE(amount_pd_deposit_2m/NULLIF(amount_credited_2m, 0), 0) as ratio_pd_amt_total_credit_2m,
COALESCE(amount_ach_debit_1m/NULLIF(amount_debited_1m, 0), 0) as ratio_ach_debit_total_debit_1m, COALESCE(amount_ach_debit_2m/NULLIF(amount_debited_2m, 0), 0) as ratio_ach_debit_total_debit_2m,
COALESCE(amount_ach_credit_1m/NULLIF(amount_credited_1m, 0), 0) as ratio_ach_credit_total_credit_1m, COALESCE(amount_ach_credit_2m/NULLIF(amount_credited_2m, 0), 0) as ratio_ach_credit_total_credit_2m,
COALESCE(amount_card_withdrawal_1m/NULLIF(amount_debited_1m, 0), 0) as ratio_card_withdrawal_total_debit_1m, COALESCE(amount_card_withdrawal_2m/NULLIF(amount_debited_2m, 0), 0) as ratio_card_withdrawal_total_debit_2m,
COALESCE(amount_mrdc_credit_1m/NULLIF(amount_credited_1m, 0), 0) as ratio_mrdc_credit_total_credit_1m, COALESCE(amount_mrdc_credit_2m/NULLIF(amount_credited_2m, 0), 0) as ratio_mrdc_credit_total_credit_2m,

count_pd_deposit_1m/(distinct_credit_txns_1m + 1) as ratio_pd_freq_total_credit_1m, count_pd_deposit_2m/(distinct_credit_txns_2m + 1) as ratio_pd_freq_total_credit_2m,
count_ach_debit_1m/(distinct_debit_txns_1m + 1) as ratio_ach_debit_freq_total_debit_1m, count_ach_debit_2m/(distinct_debit_txns_2m + 1) as ratio_ach_debit_freq_total_debit_2m,
count_ach_credit_1m/(distinct_credit_txns_1m + 1) as ratio_ach_credit_freq_total_credit_1m, count_ach_credit_2m/(distinct_credit_txns_2m + 1) as ratio_ach_credit_freq_total_credit_2m,
count_card_withdrawal_1m/(distinct_debit_txns_1m + 1) as ratio_card_withdrawal_freq_total_debit_1m, count_card_withdrawal_2m/(distinct_debit_txns_2m + 1) as ratio_card_withdrawal_freq_total_debit_2m,
count_mrdc_credit_1m/(distinct_credit_txns_1m + 1) as ratio_mrdc_credit_freq_total_credit_1m, count_mrdc_credit_2m/(distinct_credit_txns_2m + 1) as ratio_mrdc_credit_freq_total_credit_2m,

amount_ach_debit_1m/(amount_ach_credit_1m + 1) as ratio_ach_debit_credit_1m, amount_ach_debit_2m/(amount_ach_credit_2m + 1) as ratio_ach_debit_credit_2m,
count_ach_debit_1m/(count_ach_credit_1m + 1) as ratio_ach_debit_credit_freq_1m, count_ach_debit_2m/(count_ach_credit_2m + 1) as ratio_ach_debit_credit_freq_2m,
amount_ach_debit_1m/(count_ach_debit_1m + 1) as ach_d_size_1m, amount_ach_debit_2m/(count_ach_debit_2m + 1) as ach_d_size_2m,
amount_ach_credit_1m/(count_ach_credit_1m + 1) as ach_c_size_1m, amount_ach_credit_2m/(count_ach_credit_2m + 1) as ach_c_size_2m

FROM (select distinct business_id, due_date, min(dateadd(month,-1,due_date)) over(partition by business_id) as min_created_at from PROD_DB.ADHOC.LENDING_DPD_DATA_PILOT_GA) pay
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_BUSINESSES" biz on pay.business_id = biz.business_id
-- left join (select LENDING_BUSINESS_ID, min(created_at) as min_created_at, min(due_date) as first_due_date from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES"
-- group by 1) p on biz.id = p.lending_business_id
left join FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" t on pay.business_id = t.business_id
where t.status = 'active'  AND  daysFromDueDate <=180 AND daysFromDueDate > 0 and DaysSinceLoanTaken >= 0
-- and pay.lending_business_id = '48cd97e3-f301-4958-bd0d-703ec133fb0b' 
order by transaction_date, pay.due_date
)
group by 1,2


""")

In [7]:
txn_df.head()

,business_id,due_date,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,count_card_withdrawal_1m,count_mrdc_credit_1m,count_pd_deposit_2m,count_ach_debit_2m,count_ach_credit_2m,count_card_withdrawal_2m,count_mrdc_credit_2m,amount_pd_deposit_1m,amount_ach_debit_1m,amount_ach_credit_1m,amount_card_withdrawal_1m,amount_mrdc_credit_1m,amount_pd_deposit_2m,amount_ach_debit_2m,amount_ach_credit_2m,amount_card_withdrawal_2m,amount_mrdc_credit_2m,pd_deposit_size_1m,ach_debit_size_1m,ach_credit_size_1m,card_withdrawal_size_1m,mrdc_credit_size_1m,pd_deposit_size_2m,ach_debit_size_2m,ach_credit_size_2m,card_withdrawal_size_2m,mrdc_credit_size_2m,amount_credited_1m,distinct_credit_txns_1m,credit_size_1m,median_amount_credited_1m,stddev_amount_credited_1m,amount_credited_2m,distinct_credit_txns_2m,credit_size_2m,median_amount_credited_2m,stddev_amount_credited_2m,amount_debited_1m,distinct_debit_txns_1m,debit_size_1m,median_amount_debited_1m,stddev_amount_debited_1m,amount_debited_2m,distinct_debit_txns_2m,debit_size_2m,median_amount_debited_2m,stddev_amount_debited_2m,ratio_debit_credit_1m,ratio_debit_credit_frequency_1m,ratio_debit_credit_2m,ratio_debit_credit_frequency_2m,ratio_pd_amt_total_credit_1m,ratio_ach_debit_total_debit_1m,ratio_ach_credit_total_credit_1m,ratio_card_withdrawal_total_debit_1m,ratio_mrdc_credit_total_credit_1m,ratio_pd_amt_total_credit_2m,ratio_ach_debit_total_debit_2m,ratio_ach_credit_total_credit_2m,ratio_card_withdrawal_total_debit_2m,ratio_mrdc_credit_total_credit_2m,ratio_pd_freq_total_credit_1m,ratio_ach_debit_freq_total_debit_1m,ratio_ach_credit_freq_total_credit_1m,ratio_card_withdrawal_freq_total_debit_1m,ratio_mrdc_credit_freq_total_credit_1m,ratio_pd_freq_total_credit_2m,ratio_ach_debit_freq_total_debit_2m,ratio_ach_credit_freq_total_credit_2m,ratio_card_withdrawal_freq_total_debit_2m,ratio_mrdc_credit_freq_total_credit_2m,ratio_ach_debit_credit_1m,ratio_ach_debit_credit_2m,ratio_ach_debit_credit_freq_1m,ratio_ach_debit_credit_freq_2m,ach_d_size_1m,ach_d_size_2m,ach_c_size_1m,ach_c_size_2m
0,114457cb-957e-437f-a7ce-bc8ea43bcf5b,2022-07-22,5.0,2.0,5.0,36.0,0.0,6.0,6.0,6.0,42.0,0.0,381.42,153.16,381.42,933.97,0.00,438.77,529.31,438.77,1599.21,0.00,63.570000,51.053333,63.570000,25.242432,0.00000,62.681429,75.615714,62.681429,37.190930,0.000000,781.42,6.0,111.631429,0.0,56.265891,2438.77,8.0,270.974444,0.0,215.688435,1771.96,40.0,43.218537,9.975,78.855042,2813.35,50.0,55.163725,19.04,85.966454,2.264717,5.714286,1.153121,5.555556,0.488111,0.086435,0.488111,0.527083,0.000000,0.179914,0.188142,0.179914,0.568436,0.000000,0.714286,0.048780,0.714286,0.878049,0.000000,0.666667,0.117647,0.666667,0.823529,0.000000,0.400502,1.203606,0.333333,0.857143,51.053333,75.615714,63.570000,62.681429
1,2a364bf3-30f1-4054-86c2-2d3369f3bb89,2022-08-13,0.0,2.0,22.0,8.0,0.0,0.0,7.0,42.0,24.0,1.0,0.00,1647.84,10916.33,3003.15,0.00,0.00,3493.42,19854.89,22781.80,19000.00,0.000000,549.280000,474.623043,333.683333,0.00000,0.000000,436.677500,461.741628,911.272000,9500.000000,12016.33,23.0,500.680417,0.0,274.331438,41331.53,46.0,879.394255,146.3,2037.006592,5559.32,11.0,463.276667,0.000,235.822204,39091.88,34.0,1116.910857,0.00,1493.834533,0.462609,0.458333,0.945790,0.723404,0.000000,0.296410,0.908458,0.540201,0.000000,0.000000,0.089364,0.480381,0.582776,0.459697,0.000000,0.166667,0.916667,0.666667,0.000000,0.000000,0.200000,0.893617,0.685714,0.021277,0.150938,0.175939,0.086957,0.162791,549.280000,436.677500,474.623043,461.741628
2,3e48b368-a42f-4358-a0c6-ac86074ece0d,2022-09-11,10.0,6.0,16.0,200.0,7.0,26.0,14.0,32.0,325.0,13.0,1289.28,551.07,3834.68,10150.86,2031.99,4425.61,1572.74,6971.01,14081.24,3463.98,117.207273,78.724286,225.569412,50.501791,253.99875,163.911481,104.849333,211.242727,43.193988,247.427143,12151.86,26.0,450.068889,0.0,180.996127,17123.18,50.0,335.748627,0.0,187.426323,11045.27,208.0,52.848182,0.000,121.423027,17089.82,348.0,48.967966,7.01,129.676306,0.908862,7.703704,0.997993,6.823529,0.106097,0.049892,0.3

In [8]:
txn_df.shape

(4997, 84)

In [9]:
rb_df = querySno(r"""
with txns_post_lending as (
select  pay.business_id, pay.due_date, t.transaction_date,pay.min_created_at, amount, type, t.status, running_balance, 
datediff(day, pay.min_created_at, TRANSACTION_DATE) AS daySinceLoanTaken,
datediff(day, TRANSACTION_DATE, pay.due_date) AS daysFromDueDate
FROM (select distinct business_id, due_date, min(dateadd(month,-1,due_date)) over(partition by business_id) as min_created_at from PROD_DB.ADHOC.LENDING_DPD_DATA_PILOT_GA) pay
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_BUSINESSES" biz on pay.business_id = biz.business_id
-- left join (select lending_business_id, min(created_at) as min_created_at from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES" group by 1) p 
-- on biz.id = p.id
left join FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" t on t.business_id = pay.business_id
where t.status = 'active'  AND  daySinceLoanTaken <=180 AND daySinceLoanTaken > 0 and daysFromDueDate > 0
-- and pay.lending_business_id = 'cf6d8b20-574f-4362-b1f5-274bb2231c05'
order by due_date
)
,cal_dates_1 AS (
			SELECT DATEADD(DAY, SEQ4(), '2021-06-01') AS CAL_DATE
			 FROM TABLE(GENERATOR(ROWCOUNT=>960)) 
			 ORDER BY CAL_DATE DESC),
biz_dates_1 AS (
			SELECT a.BUSINESS_ID, MIN(a.transaction_date) AS FIRST_TXN_DATE
			from FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS"  a 
			GROUP BY a.BUSINESS_ID),
PILOT_BIZ_DATES_TEMP AS (
		SELECT *
		FROM  cal_dates_1 A LEFT JOIN
		biz_dates_1 B ON
		date(A.CAL_DATE) >= date(B.FIRST_TXN_DATE)
		ORDER BY CAL_DATE ASC)
,PILOT_BIZ_TXN_TEMP AS (
		SELECT   Business_id ,TRANSACTION_DATE,running_balance
		FROM
		        (SELECT *, RANK () OVER (PARTITION BY BUSINESS_ID,TRANSACTION_DATE ORDER BY TRANSACTION_DATE DESC) RANKS
		         FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" 
                 where 1=1
                 and status='active'
		         )
		WHERE RANKS=1
		order by Business_id, TRANSACTION_DATE)
,PILOT_DAILY_BALANCES_1  AS (
		SELECT BUSINESS_ID,FIRST_TXN_DATE, CAL_DATE,TRANSACTION_DATE,RUNNING_BALANCE,
		first_value(RUNNING_BALANCE) OVER (PARTITION BY BUSINESS_ID,GROUPER ORDER BY cal_date asc, TRANSACTION_DATE desc, RUNNING_BALANCE asc nulls last) as RUNNING_BALANCE_2
		FROM (
		SELECT A.BUSINESS_ID,A.CAL_DATE,A.FIRST_TXN_DATE,
		        B.TRANSACTION_DATE,B.RUNNING_BALANCE,
		         COUNT(running_balance) OVER (PARTITION BY a.BUSINESS_ID ORDER BY cal_date asc) as grouper
		        FROM PILOT_BIZ_DATES_TEMP A LEFT JOIN
		        PILOT_BIZ_TXN_TEMP B ON
		        A.BUSINESS_ID=B.BUSINESS_ID AND
		        date(A.CAL_DATE)= date(B.TRANSACTION_DATE)
		        ORDER BY A.CAL_DATE ASC)
		ORDER BY BUSINESS_ID,CAL_DATE)
,final_daily_running_balance AS (
		 SELECT distinct a.business_id, b.due_date, datediff(day, a.cal_date, b.due_date) as daysFromDueDate, cal_date, running_balance_2 AS running_balance_daily
		           FROM  PILOT_DAILY_BALANCES_1  a join txns_post_lending b 
		           on a.business_id = b.business_id
		          WHERE daysFromDueDate > 0 AND daysFromDueDate <= 180
		ORDER BY a.business_id, daysFromDueDate)
,oneMonth_rb AS (	
				SELECT business_id, due_date, min(cal_date) as min_txn_date_1mo_rb, max(cal_date) as max_txn_date_1mo_rb,
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_1m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_1m,
				               AVG(running_balance_daily) as Avg_running_balance_1m,
  							   Median(running_balance_daily) as Median_running_balance_1m,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_1m
				FROM final_daily_running_balance
				WHERE daysFromDueDate <=30
				GROUP BY 1,2)
,twoMonth_rb AS (	
				SELECT business_id, due_date, min(cal_date) as min_txn_date_2mo_rb, max(cal_date) as max_txn_date_2mo_rb,
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_2m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_2m,
				               AVG(running_balance_daily) as Avg_running_balance_2m,
  							   Median(running_balance_daily) as Median_running_balance_2m,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_2m
				FROM final_daily_running_balance
				WHERE daysFromDueDate <=60
				GROUP BY 1,2)
				
,second_last_month_rb as (	
				SELECT business_id, due_date, 
				               AVG(running_balance_daily) as Avg_running_balance_2nd_m,
  							   Median(running_balance_daily) as Median_running_balance_2nd_m,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_2nd_m
				FROM final_daily_running_balance
				WHERE daysFromDueDate <=60 and daysFromDueDate > 30
				GROUP BY 1,2)
-- select count(*) from (
select a.business_id, a.due_date, od_count_1m, zero_balance_count_1m, Avg_running_balance_1m, Median_running_balance_1m, STDDEV_running_balance_1m,
od_count_2m, zero_balance_count_2m, Avg_running_balance_2m, Median_running_balance_2m, STDDEV_running_balance_2m, Avg_running_balance_2nd_m, Median_running_balance_2nd_m, STDDEV_running_balance_2nd_m,
Avg_running_balance_1m/(Avg_running_balance_2nd_m + 1) as ratio_avg_running_balance_1M_2M
from oneMonth_rb a left join twoMonth_rb b on a.business_id = b.business_id and a.due_date = b.due_date
left join second_last_month_rb c on a.business_id = c.business_id and a.due_date = c.due_date
-- )
"""
)

In [10]:
rb_df.head()

,business_id,due_date,od_count_1m,zero_balance_count_1m,avg_running_balance_1m,median_running_balance_1m,stddev_running_balance_1m,od_count_2m,zero_balance_count_2m,avg_running_balance_2m,median_running_balance_2m,stddev_running_balance_2m,avg_running_balance_2nd_m,median_running_balance_2nd_m,stddev_running_balance_2nd_m,ratio_avg_running_balance_1m_2m
0,ad83cb39-0e5c-4e2e-b4c8-320a98010b3c,2022-11-01,25,22,308.321292,64.77,705.669042,25,22,347.474303,64.77,775.249045,963.481667,375.780,1370.322649,0.319676
1,1eb68c4c-ac56-4c38-97b9-6686accb5f34,2023-03-23,0,0,10492.407848,9833.71,1594.991972,0,0,10854.589028,9833.71,1964.289303,14838.582000,14459.625,995.639406,0.707055
2,2253a00b-c479-460d-b4b9-26110517a2e7,2023-03-27,0,0,993.017883,1102.08,271.081306,0,0,956.153343,1102.08,301.784880,555.558667,456.030,331.732458,1.784211
3,a9a77d6b-cbd2-4ee3-9d97-3ba57aea0f4c,2023-03-18,0,0,8769.878060,8633.97,697.363743,0,0,8639.283589,8633.97,819.168762,7180.978667,7219.670,651.656621,1.221095
4,ff34ba0d-908c-4a5e-bc07-1f67c30925db,2023-03-31,0,0,1166.306242,730.91,1068.222187,0,0,1485.272869,730.91,1536.093006,4908.848000,5368.075,1635.113988,0.237544


In [11]:
rb_df.shape

(4994, 16)

In [12]:
mom_df = querySno(r"""
select * from 
(
select business_id, due_date, credit_limit, avg(amount_drawn_1m_prev)amount_drawn_1m_prev, avg(amount_drawn_2m_prev)amount_drawn_2m_prev, avg(amount_cashin_1m_prev)amount_cashin_1m_prev, 
avg(amount_cashin_2m_prev)amount_cashin_2m_prev, avg(distinct_draws_1m_prev)distinct_draws_1m_prev, avg(distinct_draws_2m_prev)distinct_draws_2m_prev, avg(distinct_cashins_1m_prev)distinct_cashins_1m_prev, avg(distinct_cashins_2m_prev)distinct_cashins_2m_prev,
avg(ratio_amt_drawn_mom)ratio_amt_drawn_mom, avg(ratio_amt_cashin_mom)ratio_amt_cashin_mom, avg(ratio_count_drawn_mom)ratio_count_drawn_mom, avg(ratio_count_cashin_mom)ratio_count_cashin_mom
from (
select b.lending_business_id,a.business_id, due_date,b.created_at,credit_limit/100 as credit_limit , datediff(month, b.created_at, due_date) as txn_month, b.amount, type,
sum(case when txn_month = 0 and type = 'draw' then amount/100 else 0 end) over(partition by b.lending_business_id, due_date) as amount_drawn_1m_prev,
sum(case when txn_month = 1 and type = 'draw' then amount/100 else 0 end) over(partition by b.lending_business_id, due_date) as amount_drawn_2m_prev,
sum(case when txn_month = 0 and type = 'cash_in' then amount/100 else 0 end) over(partition by b.lending_business_id, due_date) as amount_cashin_1m_prev,
sum(case when txn_month = 1 and type = 'cash_in' then amount/100 else 0 end) over(partition by b.lending_business_id, due_date) as amount_cashin_2m_prev,

count(distinct case when txn_month = 0 and type = 'draw' then trace_number end) over(partition by b.lending_business_id, due_date) as distinct_draws_1m_prev,
count(distinct case when txn_month = 1 and type = 'draw' then trace_number end) over(partition by b.lending_business_id, due_date) as distinct_draws_2m_prev,
count(distinct case when txn_month = 0 and type = 'cash_in' then trace_number end) over(partition by b.lending_business_id, due_date) as distinct_cashins_1m_prev,
count(distinct case when txn_month = 1 and type = 'cash_in' then trace_number end) over(partition by b.lending_business_id, due_date) as distinct_cashins_2m_prev,

amount_drawn_1m_prev/(amount_drawn_2m_prev + 1) as ratio_amt_drawn_mom, 
amount_cashin_1m_prev/(amount_cashin_2m_prev + 1) as ratio_amt_cashin_mom, 
distinct_draws_1m_prev/(distinct_draws_2m_prev + 1) as ratio_count_drawn_mom, 
distinct_cashins_1m_prev/(distinct_cashins_2m_prev + 1) as ratio_count_cashin_mom

from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_TRANSACTIONS" b 
join 
(select distinct business_id, due_date from PROD_DB.ADHOC.LENDING_DPD_DATA_PILOT_GA) a
on a.business_id = b.business_id
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_BUSINESSES" c on b.lending_business_id = c.id
where txn_month >= 0 and txn_month <2
-- and b.lending_business_id = '005da9f7-5939-4d37-bac8-4956a2a9e419'
order by due_date
)
group by 1,2,3
order by 1,2,3
)

UNION

(
select business_id, due_date, credit_limit, avg(amount_drawn_1m_prev)amount_drawn_1m_prev, avg(amount_drawn_2m_prev)amount_drawn_2m_prev, avg(amount_cashin_1m_prev)amount_cashin_1m_prev, 
avg(amount_cashin_2m_prev)amount_cashin_2m_prev, avg(distinct_draws_1m_prev)distinct_draws_1m_prev, avg(distinct_draws_2m_prev)distinct_draws_2m_prev, avg(distinct_cashins_1m_prev)distinct_cashins_1m_prev, avg(distinct_cashins_2m_prev)distinct_cashins_2m_prev,
avg(ratio_amt_drawn_mom)ratio_amt_drawn_mom, avg(ratio_amt_cashin_mom)ratio_amt_cashin_mom, avg(ratio_count_drawn_mom)ratio_count_drawn_mom, avg(ratio_count_cashin_mom)ratio_count_cashin_mom
from (
select p.business_id, b.due_date,a.created_at,p.credit_limit , datediff(month, t.transaction_date, b.due_date) as txn_month, t.amount, a.action_type,
sum(case when txn_month = 0 and action_type = 'DRAW' then t.amount else 0 end) over(partition by b.business_id, b.due_date) as amount_drawn_1m_prev,
sum(case when txn_month = 1 and action_type = 'DRAW' then t.amount else 0 end) over(partition by b.business_id, b.due_date) as amount_drawn_2m_prev,
sum(case when txn_month = 0 and action_type = 'CASHIN' then -t.amount else 0 end) over(partition by b.business_id, b.due_date) as amount_cashin_1m_prev,
sum(case when txn_month = 1 and action_type = 'CASHIN' then -t.amount else 0 end) over(partition by b.business_id, b.due_date) as amount_cashin_2m_prev,

count(distinct case when txn_month = 0 and action_type = 'DRAW' then a.transaction_id end) over(partition by b.business_id, b.due_date) as distinct_draws_1m_prev,
count(distinct case when txn_month = 1 and action_type = 'DRAW' then a.transaction_id end) over(partition by b.business_id, b.due_date) as distinct_draws_2m_prev,
count(distinct case when txn_month = 0 and action_type = 'CASHIN' then a.transaction_id end) over(partition by b.business_id, b.due_date) as distinct_cashins_1m_prev,
count(distinct case when txn_month = 1 and action_type = 'CASHIN' then a.transaction_id end) over(partition by b.business_id, b.due_date) as distinct_cashins_2m_prev,

amount_drawn_1m_prev/(amount_drawn_2m_prev + 1) as ratio_amt_drawn_mom, 
amount_cashin_1m_prev/(amount_cashin_2m_prev + 1) as ratio_amt_cashin_mom, 
distinct_draws_1m_prev/(distinct_draws_2m_prev + 1) as ratio_count_drawn_mom, 
distinct_cashins_1m_prev/(distinct_cashins_2m_prev + 1) as ratio_count_cashin_mom

from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LOAN_PRODUCT_TRANSACTIONS a 
join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LOAN_PRODUCTS" p on a.product_id = p.id
join 
(select distinct business_id, due_date from PROD_DB.ADHOC.LENDING_DPD_DATA_PILOT_GA
where due_date < date('2023-01-01')) b 
on p.business_id = b.business_id
left join prod_db.data.transactions t on a.transaction_id = t.transaction_id
where txn_month >= 0 and txn_month <2
-- and b.lending_business_id = '005da9f7-5939-4d37-bac8-4956a2a9e419'
order by due_date
)
group by 1,2,3
order by 1,2,3
)
"""
)

In [13]:
mom_df.head()

,business_id,due_date,credit_limit,amount_drawn_1m_prev,amount_drawn_2m_prev,amount_cashin_1m_prev,amount_cashin_2m_prev,distinct_draws_1m_prev,distinct_draws_2m_prev,distinct_cashins_1m_prev,distinct_cashins_2m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom
0,0feebcf0-4e46-46fd-b8c9-dd2d81f2f339,2023-02-27,5600.0,0.0,800.0,155.33,0.00,0.0,1.0,1.0,0.0,0.000000,155.330000,0.0,1.000
1,1117adea-76d9-46c1-bfc6-6ec754267cea,2023-01-30,10000.0,4000.0,2000.0,1149.99,0.00,1.0,1.0,1.0,0.0,1.999000,1149.990000,0.5,1.000
2,12b3f26a-767b-46b3-a49f-fa6725745a37,2023-02-06,5000.0,0.0,2000.0,93.89,2396.42,0.0,2.0,1.0,7.0,0.000000,0.039163,0.0,0.125
3,176ca682-79fb-47bd-b08e-10cc4155a8c5,2023-03-28,2900.0,500.0,0.0,300.00,189.16,1.0,0.0,1.0,1.0,500.000000,1.577619,1.0,0.500
4,100e66f2-6836-4801-ab35-c43d3b01840a,2023-01-31,7000.0,4100.0,1000.0,668.66,668.66,1.0,1.0,1.0,1.0,4.095904,0.998507,0.5,0.500


In [14]:
mom_df.shape

(5117, 15)

In [15]:
mom_df = mom_df.drop_duplicates(subset = ['business_id','due_date'])

In [16]:
print (rev_df.shape)
print (txn_df.shape)
print (rb_df.shape)
print (mom_df.shape)

(4967, 6)
(4997, 84)
(4994, 16)
(4957, 15)


In [17]:
df1 = txn_df.merge(rb_df,how='inner',left_on = ['business_id','due_date'], right_on = ['business_id','due_date'])
df2 = df1.merge(rev_df,how='inner',left_on = ['business_id','due_date'], right_on = ['business_id','due_date'])
df = df2.merge(mom_df,how='inner',left_on = ['business_id','due_date'], right_on = ['business_id','due_date'])

In [18]:
df.shape

(4925, 115)

In [19]:
df.head()

,business_id,due_date,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,count_card_withdrawal_1m,count_mrdc_credit_1m,count_pd_deposit_2m,count_ach_debit_2m,count_ach_credit_2m,count_card_withdrawal_2m,count_mrdc_credit_2m,amount_pd_deposit_1m,amount_ach_debit_1m,amount_ach_credit_1m,amount_card_withdrawal_1m,amount_mrdc_credit_1m,amount_pd_deposit_2m,amount_ach_debit_2m,amount_ach_credit_2m,amount_card_withdrawal_2m,amount_mrdc_credit_2m,pd_deposit_size_1m,ach_debit_size_1m,ach_credit_size_1m,card_withdrawal_size_1m,mrdc_credit_size_1m,pd_deposit_size_2m,ach_debit_size_2m,ach_credit_size_2m,card_withdrawal_size_2m,mrdc_credit_size_2m,amount_credited_1m,distinct_credit_txns_1m,credit_size_1m,median_amount_credited_1m,stddev_amount_credited_1m,amount_credited_2m,distinct_credit_txns_2m,credit_size_2m,median_amount_credited_2m,stddev_amount_credited_2m,amount_debited_1m,distinct_debit_txns_1m,debit_size_1m,median_amount_debited_1m,stddev_amount_debited_1m,amount_debited_2m,distinct_debit_txns_2m,debit_size_2m,median_amount_debited_2m,stddev_amount_debited_2m,ratio_debit_credit_1m,ratio_debit_credit_frequency_1m,ratio_debit_credit_2m,ratio_debit_credit_frequency_2m,ratio_pd_amt_total_credit_1m,ratio_ach_debit_total_debit_1m,ratio_ach_credit_total_credit_1m,ratio_card_withdrawal_total_debit_1m,ratio_mrdc_credit_total_credit_1m,ratio_pd_amt_total_credit_2m,ratio_ach_debit_total_debit_2m,ratio_ach_credit_total_credit_2m,ratio_card_withdrawal_total_debit_2m,ratio_mrdc_credit_total_credit_2m,ratio_pd_freq_total_credit_1m,ratio_ach_debit_freq_total_debit_1m,ratio_ach_credit_freq_total_credit_1m,ratio_card_withdrawal_freq_total_debit_1m,ratio_mrdc_credit_freq_total_credit_1m,ratio_pd_freq_total_credit_2m,ratio_ach_debit_freq_total_debit_2m,ratio_ach_credit_freq_total_credit_2m,ratio_card_withdrawal_freq_total_debit_2m,ratio_mrdc_credit_freq_total_credit_2m,ratio_ach_debit_credit_1m,ratio_ach_debit_credit_2m,ratio_ach_debit_credit_freq_1m,ratio_ach_debit_credit_freq_2m,ach_d_size_1m,ach_d_size_2m,ach_c_size_1m,ach_c_size_2m,od_count_1m,zero_balance_count_1m,avg_running_balance_1m,median_running_balance_1m,stddev_running_balance_1m,od_count_2m,zero_balance_count_2m,avg_running_balance_2m,median_running_balance_2m,stddev_running_balance_2m,avg_running_balance_2nd_m,median_running_balance_2nd_m,stddev_running_balance_2nd_m,ratio_avg_running_balance_1m_2m,revenue_past_1m,revenue_past_2m,total_revenue_2m,ratio_revenue_1m_2m,credit_limit,amount_drawn_1m_prev,amount_drawn_2m_prev,amount_cashin_1m_prev,amount_cashin_2m_prev,distinct_draws_1m_prev,distinct_draws_2m_prev,distinct_cashins_1m_prev,distinct_cashins_2m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom
0,114457cb-957e-437f-a7ce-bc8ea43bcf5b,2022-07-22,5.0,2.0,5.0,36.0,0.0,6.0,6.0,6.0,42.0,0.0,381.42,153.16,381.42,933.97,0.00,438.77,529.31,438.77,1599.21,0.00,63.570000,51.053333,63.570000,25.242432,0.00000,62.681429,75.615714,62.681429,37.190930,0.000000,781.42,6.0,111.631429,0.0,56.265891,2438.77,8.0,270.974444,0.0,215.688435,1771.96,40.0,43.218537,9.975,78.855042,2813.35,50.0,55.163725,19.04,85.966454,2.264717,5.714286,1.153121,5.555556,0.488111,0.086435,0.488111,0.527083,0.000000,0.179914,0.188142,0.179914,0.568436,0.000000,0.714286,0.048780,0.714286,0.878049,0.000000,0.666667,0.117647,0.666667,0.823529,0.000000,0.400502,1.203606,0.333333,0.857143,51.053333,75.615714,63.570000,62.681429,65,484,-19.239094,0.000,272.725779,65,484,7.581142,0.00,305.006155,520.741667,449.425,421.984920,-0.036875,245.52,1015.50,1261.02,0.241535,2000.0,0.0,2000.0,726.66,0.00,0.0,2.0,2.0,0.0,0.000000,726.660000,0.000000,2.000000
1,2a364bf3-30f1-4054-86c2-2d3369f3bb89,2022-08-13,0.0,2.0,22.0,8.0,0.0,0.0,7.0,42.0,24.0,1.0,0.00,1647.84,10916.33,3003.15,0.00,0.00,3493.42,19854.89,22781.80,19000.00,0.000000,549.280000,474.623043,333.683333,0.00000,0.000000,436.677500,461.741628,911.272000,9500.000000,12016.33,23.0,500.680417,0.0,274.331438,41331.53,46.0,879.39425

In [20]:
df.describe()

,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,count_card_withdrawal_1m,count_mrdc_credit_1m,count_pd_deposit_2m,count_ach_debit_2m,count_ach_credit_2m,count_card_withdrawal_2m,count_mrdc_credit_2m,amount_pd_deposit_1m,amount_ach_debit_1m,amount_ach_credit_1m,amount_card_withdrawal_1m,amount_mrdc_credit_1m,amount_pd_deposit_2m,amount_ach_debit_2m,amount_ach_credit_2m,amount_card_withdrawal_2m,amount_mrdc_credit_2m,pd_deposit_size_1m,ach_debit_size_1m,ach_credit_size_1m,card_withdrawal_size_1m,mrdc_credit_size_1m,pd_deposit_size_2m,ach_debit_size_2m,ach_credit_size_2m,card_withdrawal_size_2m,mrdc_credit_size_2m,amount_credited_1m,distinct_credit_txns_1m,credit_size_1m,median_amount_credited_1m,stddev_amount_credited_1m,amount_credited_2m,distinct_credit_txns_2m,credit_size_2m,median_amount_credited_2m,stddev_amount_credited_2m,amount_debited_1m,distinct_debit_txns_1m,debit_size_1m,median_amount_debited_1m,stddev_amount_debited_1m,amount_debited_2m,distinct_debit_txns_2m,debit_size_2m,median_amount_debited_2m,stddev_amount_debited_2m,ratio_debit_credit_1m,ratio_debit_credit_frequency_1m,ratio_debit_credit_2m,ratio_debit_credit_frequency_2m,ratio_pd_amt_total_credit_1m,ratio_ach_debit_total_debit_1m,ratio_ach_credit_total_credit_1m,ratio_card_withdrawal_total_debit_1m,ratio_mrdc_credit_total_credit_1m,ratio_pd_amt_total_credit_2m,ratio_ach_debit_total_debit_2m,ratio_ach_credit_total_credit_2m,ratio_card_withdrawal_total_debit_2m,ratio_mrdc_credit_total_credit_2m,ratio_pd_freq_total_credit_1m,ratio_ach_debit_freq_total_debit_1m,ratio_ach_credit_freq_total_credit_1m,ratio_card_withdrawal_freq_total_debit_1m,ratio_mrdc_credit_freq_total_credit_1m,ratio_pd_freq_total_credit_2m,ratio_ach_debit_freq_total_debit_2m,ratio_ach_credit_freq_total_credit_2m,ratio_card_withdrawal_freq_total_debit_2m,ratio_mrdc_credit_freq_total_credit_2m,ratio_ach_debit_credit_1m,ratio_ach_debit_credit_2m,ratio_ach_debit_credit_freq_1m,ratio_ach_debit_credit_freq_2m,ach_d_size_1m,ach_d_size_2m,ach_c_size_1m,ach_c_size_2m,od_count_1m,zero_balance_count_1m,avg_running_balance_1m,median_running_balance_1m,stddev_running_balance_1m,od_count_2m,zero_balance_count_2m,avg_running_balance_2m,median_running_balance_2m,stddev_running_balance_2m,avg_running_balance_2nd_m,median_running_balance_2nd_m,stddev_running_balance_2nd_m,ratio_avg_running_balance_1m_2m,revenue_past_1m,revenue_past_2m,total_revenue_2m,ratio_revenue_1m_2m,credit_limit,amount_drawn_1m_prev,amount_drawn_2m_prev,amount_cashin_1m_prev,amount_cashin_2m_prev,distinct_draws_1m_prev,distinct_draws_2m_prev,distinct_cashins_1m_prev,distinct_cashins_2m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom
count,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4

In [21]:
null_df = pd.DataFrame(df.isnull().sum().sort_values(ascending=False),columns = ['null_count'])
null_df = null_df[null_df['null_count']>0]

In [22]:
null_df.shape

(0, 1)

In [23]:
# df = df.fillna(0)

In [24]:
base_df = querySno(r"""
select * from PROD_DB.ADHOC.LENDING_DPD_DATA_PILOT_GA
""")

In [25]:
base_df.shape

(5017, 6)

In [26]:
base_df.sample(5)

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
1970,34a3025a-2f69-4cc2-b4c4-f060294fb0a7,2023-03-27,0,0,0,2
3678,dcb4b3ae-41b7-439e-9155-c55a784b4e23,2022-09-23,0,0,0,1
1653,79edc428-7fc5-434e-9f2e-5b24240af02b,2022-12-31,0,0,0,4
873,3f5d860e-7a07-4ac3-a1c8-599dfc969e2a,2023-02-27,0,0,0,1
3509,018c84d4-87f6-4250-a8fe-d087ea9ce961,2022-10-02,1,1,1,1


In [27]:
base_df = base_df.drop_duplicates(subset = ['business_id','due_date'])

In [28]:
base_df['DPD_plus_15'] = base_df['DPD_plus_15'].astype(str).astype(int)
base_df['DPD_plus_3'] = base_df['DPD_plus_3'].astype(str).astype(int)
base_df['emi_no'] = base_df['emi_no'].astype(str).astype(int)

In [29]:
base_df.head()

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
0,c616b39b-aef7-4156-973a-08278aa74536,2023-04-03,0,0,0,1
1,cc861415-fd34-4654-8b97-dab0794938dc,2023-04-03,0,0,0,1
2,4d7e9b73-dd94-4fdc-9611-51ec641b8e19,2023-04-03,1,0,0,1
3,2e3c32e2-1f1b-4d0f-a7c2-8dcdb11c6317,2023-04-03,0,0,0,1
4,57948378-e103-4f19-972e-a911082a16c2,2023-04-03,0,0,0,1


In [30]:
b = base_df.copy()

In [31]:
base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby(['business_id'])['DPD_plus_3'].shift(-1).rename('DPD_plus_3_next'),left_index=True,right_index=True)
base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby(['business_id'])['DPD_plus_3'].shift(1).rename('DPD_plus_3_prev'),left_index=True,right_index=True)
# base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby(['business_id'])['DPD_plus_3'].shift(2).rename('DPD_plus_3_prev_prev'),left_index=True,right_index=True)
base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby(['business_id'])['DPD_plus_15'].shift(-1).rename('DPD_plus_15_next'),left_index=True,right_index=True)
base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby(['business_id'])['DPD_plus_15'].shift(1).rename('DPD_plus_15_prev'),left_index=True,right_index=True)
# base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby(['business_id'])['DPD_plus_15'].shift(2).rename('DPD_plus_15_prev_prev'),left_index=True,right_index=True)

In [32]:
base_df = base_df.fillna(0)

In [33]:
base_df['DPD_plus_3_last_2'] = base_df['DPD_plus_3_prev'] + base_df['DPD_plus_3']
base_df['DPD_plus_15_last_2'] = base_df['DPD_plus_15_prev'] + base_df['DPD_plus_15']

In [34]:
# base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby('business_id')['DPD_plus_3_prev'].cumsum().rename('DPD_plus_3_prev_cumsum'),left_index=True,right_index=True)
# base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby('business_id')['DPD_plus_15_prev'].cumsum().rename('DPD_plus_15_prev_cumsum'),left_index=True,right_index=True)

In [35]:
# base_df['DPD_plus_3_prev_ratio'] = base_df['DPD_plus_3_prev_cumsum']/(base_df['emi_no']-1)
# base_df['DPD_plus_15_prev_ratio'] = base_df['DPD_plus_15_prev_cumsum']/(base_df['emi_no']-1)
# base_df = base_df.fillna(0)

In [36]:
base_df[base_df['business_id']=='1724f6de-5ba9-45a7-917d-8a0e575b6297'].sort_values('emi_no')

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no,DPD_plus_3_next,DPD_plus_3_prev,DPD_plus_15_next,DPD_plus_15_prev,DPD_plus_3_last_2,DPD_plus_15_last_2
3273,1724f6de-5ba9-45a7-917d-8a0e575b6297,2022-09-10,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0
3916,1724f6de-5ba9-45a7-917d-8a0e575b6297,2022-09-20,0,0,0,2,0.0,0.0,0.0,0.0,0.0,0.0
4559,1724f6de-5ba9-45a7-917d-8a0e575b6297,2022-10-10,0,0,0,3,0.0,0.0,0.0,0.0,0.0,0.0
1375,1724f6de-5ba9-45a7-917d-8a0e575b6297,2023-01-21,0,0,0,4,1.0,0.0,0.0,0.0,0.0,0.0
2450,1724f6de-5ba9-45a7-917d-8a0e575b6297,2023-02-21,1,0,0,5,0.0,0.0,0.0,0.0,1.0,0.0
2796,1724f6de-5ba9-45a7-917d-8a0e575b6297,2023-03-21,0,0,0,6,0.0,1.0,0.0,0.0,1.0,0.0


In [37]:
bdf = base_df[['business_id','due_date','DPD_plus_15_next','DPD_plus_3_last_2','DPD_plus_15_prev','emi_no','DPD_plus_3']]

In [38]:
df.head()

,business_id,due_date,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,count_card_withdrawal_1m,count_mrdc_credit_1m,count_pd_deposit_2m,count_ach_debit_2m,count_ach_credit_2m,count_card_withdrawal_2m,count_mrdc_credit_2m,amount_pd_deposit_1m,amount_ach_debit_1m,amount_ach_credit_1m,amount_card_withdrawal_1m,amount_mrdc_credit_1m,amount_pd_deposit_2m,amount_ach_debit_2m,amount_ach_credit_2m,amount_card_withdrawal_2m,amount_mrdc_credit_2m,pd_deposit_size_1m,ach_debit_size_1m,ach_credit_size_1m,card_withdrawal_size_1m,mrdc_credit_size_1m,pd_deposit_size_2m,ach_debit_size_2m,ach_credit_size_2m,card_withdrawal_size_2m,mrdc_credit_size_2m,amount_credited_1m,distinct_credit_txns_1m,credit_size_1m,median_amount_credited_1m,stddev_amount_credited_1m,amount_credited_2m,distinct_credit_txns_2m,credit_size_2m,median_amount_credited_2m,stddev_amount_credited_2m,amount_debited_1m,distinct_debit_txns_1m,debit_size_1m,median_amount_debited_1m,stddev_amount_debited_1m,amount_debited_2m,distinct_debit_txns_2m,debit_size_2m,median_amount_debited_2m,stddev_amount_debited_2m,ratio_debit_credit_1m,ratio_debit_credit_frequency_1m,ratio_debit_credit_2m,ratio_debit_credit_frequency_2m,ratio_pd_amt_total_credit_1m,ratio_ach_debit_total_debit_1m,ratio_ach_credit_total_credit_1m,ratio_card_withdrawal_total_debit_1m,ratio_mrdc_credit_total_credit_1m,ratio_pd_amt_total_credit_2m,ratio_ach_debit_total_debit_2m,ratio_ach_credit_total_credit_2m,ratio_card_withdrawal_total_debit_2m,ratio_mrdc_credit_total_credit_2m,ratio_pd_freq_total_credit_1m,ratio_ach_debit_freq_total_debit_1m,ratio_ach_credit_freq_total_credit_1m,ratio_card_withdrawal_freq_total_debit_1m,ratio_mrdc_credit_freq_total_credit_1m,ratio_pd_freq_total_credit_2m,ratio_ach_debit_freq_total_debit_2m,ratio_ach_credit_freq_total_credit_2m,ratio_card_withdrawal_freq_total_debit_2m,ratio_mrdc_credit_freq_total_credit_2m,ratio_ach_debit_credit_1m,ratio_ach_debit_credit_2m,ratio_ach_debit_credit_freq_1m,ratio_ach_debit_credit_freq_2m,ach_d_size_1m,ach_d_size_2m,ach_c_size_1m,ach_c_size_2m,od_count_1m,zero_balance_count_1m,avg_running_balance_1m,median_running_balance_1m,stddev_running_balance_1m,od_count_2m,zero_balance_count_2m,avg_running_balance_2m,median_running_balance_2m,stddev_running_balance_2m,avg_running_balance_2nd_m,median_running_balance_2nd_m,stddev_running_balance_2nd_m,ratio_avg_running_balance_1m_2m,revenue_past_1m,revenue_past_2m,total_revenue_2m,ratio_revenue_1m_2m,credit_limit,amount_drawn_1m_prev,amount_drawn_2m_prev,amount_cashin_1m_prev,amount_cashin_2m_prev,distinct_draws_1m_prev,distinct_draws_2m_prev,distinct_cashins_1m_prev,distinct_cashins_2m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom
0,114457cb-957e-437f-a7ce-bc8ea43bcf5b,2022-07-22,5.0,2.0,5.0,36.0,0.0,6.0,6.0,6.0,42.0,0.0,381.42,153.16,381.42,933.97,0.00,438.77,529.31,438.77,1599.21,0.00,63.570000,51.053333,63.570000,25.242432,0.00000,62.681429,75.615714,62.681429,37.190930,0.000000,781.42,6.0,111.631429,0.0,56.265891,2438.77,8.0,270.974444,0.0,215.688435,1771.96,40.0,43.218537,9.975,78.855042,2813.35,50.0,55.163725,19.04,85.966454,2.264717,5.714286,1.153121,5.555556,0.488111,0.086435,0.488111,0.527083,0.000000,0.179914,0.188142,0.179914,0.568436,0.000000,0.714286,0.048780,0.714286,0.878049,0.000000,0.666667,0.117647,0.666667,0.823529,0.000000,0.400502,1.203606,0.333333,0.857143,51.053333,75.615714,63.570000,62.681429,65,484,-19.239094,0.000,272.725779,65,484,7.581142,0.00,305.006155,520.741667,449.425,421.984920,-0.036875,245.52,1015.50,1261.02,0.241535,2000.0,0.0,2000.0,726.66,0.00,0.0,2.0,2.0,0.0,0.000000,726.660000,0.000000,2.000000
1,2a364bf3-30f1-4054-86c2-2d3369f3bb89,2022-08-13,0.0,2.0,22.0,8.0,0.0,0.0,7.0,42.0,24.0,1.0,0.00,1647.84,10916.33,3003.15,0.00,0.00,3493.42,19854.89,22781.80,19000.00,0.000000,549.280000,474.623043,333.683333,0.00000,0.000000,436.677500,461.741628,911.272000,9500.000000,12016.33,23.0,500.680417,0.0,274.331438,41331.53,46.0,879.39425

In [132]:
dff = bdf.merge(df,how='inner', left_on = ['business_id','due_date'], right_on = ['business_id','due_date'])

In [133]:
dff.head()

,business_id,due_date,DPD_plus_15_next,DPD_plus_3_last_2,DPD_plus_15_prev,emi_no,DPD_plus_3,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,count_card_withdrawal_1m,count_mrdc_credit_1m,count_pd_deposit_2m,count_ach_debit_2m,count_ach_credit_2m,count_card_withdrawal_2m,count_mrdc_credit_2m,amount_pd_deposit_1m,amount_ach_debit_1m,amount_ach_credit_1m,amount_card_withdrawal_1m,amount_mrdc_credit_1m,amount_pd_deposit_2m,amount_ach_debit_2m,amount_ach_credit_2m,amount_card_withdrawal_2m,amount_mrdc_credit_2m,pd_deposit_size_1m,ach_debit_size_1m,ach_credit_size_1m,card_withdrawal_size_1m,mrdc_credit_size_1m,pd_deposit_size_2m,ach_debit_size_2m,ach_credit_size_2m,card_withdrawal_size_2m,mrdc_credit_size_2m,amount_credited_1m,distinct_credit_txns_1m,credit_size_1m,median_amount_credited_1m,stddev_amount_credited_1m,amount_credited_2m,distinct_credit_txns_2m,credit_size_2m,median_amount_credited_2m,stddev_amount_credited_2m,amount_debited_1m,distinct_debit_txns_1m,debit_size_1m,median_amount_debited_1m,stddev_amount_debited_1m,amount_debited_2m,distinct_debit_txns_2m,debit_size_2m,median_amount_debited_2m,stddev_amount_debited_2m,ratio_debit_credit_1m,ratio_debit_credit_frequency_1m,ratio_debit_credit_2m,ratio_debit_credit_frequency_2m,ratio_pd_amt_total_credit_1m,ratio_ach_debit_total_debit_1m,ratio_ach_credit_total_credit_1m,ratio_card_withdrawal_total_debit_1m,ratio_mrdc_credit_total_credit_1m,ratio_pd_amt_total_credit_2m,ratio_ach_debit_total_debit_2m,ratio_ach_credit_total_credit_2m,ratio_card_withdrawal_total_debit_2m,ratio_mrdc_credit_total_credit_2m,ratio_pd_freq_total_credit_1m,ratio_ach_debit_freq_total_debit_1m,ratio_ach_credit_freq_total_credit_1m,ratio_card_withdrawal_freq_total_debit_1m,ratio_mrdc_credit_freq_total_credit_1m,ratio_pd_freq_total_credit_2m,ratio_ach_debit_freq_total_debit_2m,ratio_ach_credit_freq_total_credit_2m,ratio_card_withdrawal_freq_total_debit_2m,ratio_mrdc_credit_freq_total_credit_2m,ratio_ach_debit_credit_1m,ratio_ach_debit_credit_2m,ratio_ach_debit_credit_freq_1m,ratio_ach_debit_credit_freq_2m,ach_d_size_1m,ach_d_size_2m,ach_c_size_1m,ach_c_size_2m,od_count_1m,zero_balance_count_1m,avg_running_balance_1m,median_running_balance_1m,stddev_running_balance_1m,od_count_2m,zero_balance_count_2m,avg_running_balance_2m,median_running_balance_2m,stddev_running_balance_2m,avg_running_balance_2nd_m,median_running_balance_2nd_m,stddev_running_balance_2nd_m,ratio_avg_running_balance_1m_2m,revenue_past_1m,revenue_past_2m,total_revenue_2m,ratio_revenue_1m_2m,credit_limit,amount_drawn_1m_prev,amount_drawn_2m_prev,amount_cashin_1m_prev,amount_cashin_2m_prev,distinct_draws_1m_prev,distinct_draws_2m_prev,distinct_cashins_1m_prev,distinct_cashins_2m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom
0,c616b39b-aef7-4156-973a-08278aa74536,2023-04-03,0.0,0.0,0.0,1,0,0.0,1.0,0.0,35.0,0.0,0.0,1.0,0.0,38.0,0.0,0.00,295.36,0.00,3962.06,0.0,0.00,295.36,0.00,4102.83,0.0,0.000000,147.680000,0.000000,110.057222,0.0,0.000000,147.680000,0.00000,105.200769,0.0,1800.00,2.0,600.000000,0.00,195.239512,2800.00,3.0,700.000000,0.00,244.616840,4257.42,36.0,115.065405,35.885,192.561543,4398.19,39.0,109.954750,42.995,191.137528,2.363920,12.000000,1.570221,9.750000,0.000000,0.069375,0.000000,0.930625,0.0,0.000000,0.067155,0.000000,0.932845,0.0,0.000000,0.027027,0.000000,0.945946,0.0,0.000000,0.025000,0.000000,0.950000,0.0,295.360000,295.360000,1.000000,1.000000,147.680000,147.680000,0.000000,0.00000,0,0,17737.240094,18065.85,1433.419027,0,0,17883.244097,18065.85,1689.488039,19435.753333,21236.060,2998.968352,0.912562,12494.28,0.00,12494.28,12494.280000,11600.0,1000.0,2800.0,2884.00,0.00,1.0,3.0,2.0,0.0,0.357015,2884.000000,0.25,2.000000
1,cc861415-fd34-4654-8b97-dab0794938dc,2023-04-03,0.0,0.0,0.0,1,0,0.0,14.0,4.0,31.0,0.0,0.0,15.0,4.0,31.0,0.0,0.00,10562.34,15750.00,1948.69,0.0,0.00,10902.18,15750.00,1948.69,0.0,0.000000,704.156000,3150.000000,60.896563,0.0,0.000000,681.386250,3150.0000

In [41]:
dff.shape

(4925, 120)

In [134]:
dff['emi_no'].value_counts()

1    1971
2    1376
3     662
4     331
5     315
6     270
Name: emi_no, dtype: int64

In [43]:
dff.head()

,business_id,due_date,DPD_plus_15_next,DPD_plus_3_last_2,DPD_plus_15_prev,emi_no,DPD_plus_3,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,count_card_withdrawal_1m,count_mrdc_credit_1m,count_pd_deposit_2m,count_ach_debit_2m,count_ach_credit_2m,count_card_withdrawal_2m,count_mrdc_credit_2m,amount_pd_deposit_1m,amount_ach_debit_1m,amount_ach_credit_1m,amount_card_withdrawal_1m,amount_mrdc_credit_1m,amount_pd_deposit_2m,amount_ach_debit_2m,amount_ach_credit_2m,amount_card_withdrawal_2m,amount_mrdc_credit_2m,pd_deposit_size_1m,ach_debit_size_1m,ach_credit_size_1m,card_withdrawal_size_1m,mrdc_credit_size_1m,pd_deposit_size_2m,ach_debit_size_2m,ach_credit_size_2m,card_withdrawal_size_2m,mrdc_credit_size_2m,amount_credited_1m,distinct_credit_txns_1m,credit_size_1m,median_amount_credited_1m,stddev_amount_credited_1m,amount_credited_2m,distinct_credit_txns_2m,credit_size_2m,median_amount_credited_2m,stddev_amount_credited_2m,amount_debited_1m,distinct_debit_txns_1m,debit_size_1m,median_amount_debited_1m,stddev_amount_debited_1m,amount_debited_2m,distinct_debit_txns_2m,debit_size_2m,median_amount_debited_2m,stddev_amount_debited_2m,ratio_debit_credit_1m,ratio_debit_credit_frequency_1m,ratio_debit_credit_2m,ratio_debit_credit_frequency_2m,ratio_pd_amt_total_credit_1m,ratio_ach_debit_total_debit_1m,ratio_ach_credit_total_credit_1m,ratio_card_withdrawal_total_debit_1m,ratio_mrdc_credit_total_credit_1m,ratio_pd_amt_total_credit_2m,ratio_ach_debit_total_debit_2m,ratio_ach_credit_total_credit_2m,ratio_card_withdrawal_total_debit_2m,ratio_mrdc_credit_total_credit_2m,ratio_pd_freq_total_credit_1m,ratio_ach_debit_freq_total_debit_1m,ratio_ach_credit_freq_total_credit_1m,ratio_card_withdrawal_freq_total_debit_1m,ratio_mrdc_credit_freq_total_credit_1m,ratio_pd_freq_total_credit_2m,ratio_ach_debit_freq_total_debit_2m,ratio_ach_credit_freq_total_credit_2m,ratio_card_withdrawal_freq_total_debit_2m,ratio_mrdc_credit_freq_total_credit_2m,ratio_ach_debit_credit_1m,ratio_ach_debit_credit_2m,ratio_ach_debit_credit_freq_1m,ratio_ach_debit_credit_freq_2m,ach_d_size_1m,ach_d_size_2m,ach_c_size_1m,ach_c_size_2m,od_count_1m,zero_balance_count_1m,avg_running_balance_1m,median_running_balance_1m,stddev_running_balance_1m,od_count_2m,zero_balance_count_2m,avg_running_balance_2m,median_running_balance_2m,stddev_running_balance_2m,avg_running_balance_2nd_m,median_running_balance_2nd_m,stddev_running_balance_2nd_m,ratio_avg_running_balance_1m_2m,revenue_past_1m,revenue_past_2m,total_revenue_2m,ratio_revenue_1m_2m,credit_limit,amount_drawn_1m_prev,amount_drawn_2m_prev,amount_cashin_1m_prev,amount_cashin_2m_prev,distinct_draws_1m_prev,distinct_draws_2m_prev,distinct_cashins_1m_prev,distinct_cashins_2m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom
0,c616b39b-aef7-4156-973a-08278aa74536,2023-04-03,0.0,0.0,0.0,1,0,0.0,1.0,0.0,35.0,0.0,0.0,1.0,0.0,38.0,0.0,0.00,295.36,0.00,3962.06,0.0,0.00,295.36,0.00,4102.83,0.0,0.000000,147.680000,0.000000,110.057222,0.0,0.000000,147.680000,0.00000,105.200769,0.0,1800.00,2.0,600.000000,0.00,195.239512,2800.00,3.0,700.000000,0.00,244.616840,4257.42,36.0,115.065405,35.885,192.561543,4398.19,39.0,109.954750,42.995,191.137528,2.363920,12.000000,1.570221,9.750000,0.000000,0.069375,0.000000,0.930625,0.0,0.000000,0.067155,0.000000,0.932845,0.0,0.000000,0.027027,0.000000,0.945946,0.0,0.000000,0.025000,0.000000,0.950000,0.0,295.360000,295.360000,1.000000,1.000000,147.680000,147.680000,0.000000,0.00000,0,0,17737.240094,18065.85,1433.419027,0,0,17883.244097,18065.85,1689.488039,19435.753333,21236.060,2998.968352,0.912562,12494.28,0.00,12494.28,12494.280000,11600.0,1000.0,2800.0,2884.00,0.00,1.0,3.0,2.0,0.0,0.357015,2884.000000,0.25,2.000000
1,cc861415-fd34-4654-8b97-dab0794938dc,2023-04-03,0.0,0.0,0.0,1,0,0.0,14.0,4.0,31.0,0.0,0.0,15.0,4.0,31.0,0.0,0.00,10562.34,15750.00,1948.69,0.0,0.00,10902.18,15750.00,1948.69,0.0,0.000000,704.156000,3150.000000,60.896563,0.0,0.000000,681.386250,3150.0000

In [135]:
temp = dff.groupby('business_id',as_index=False)['due_date'].min()
temp['pilot_flag'] = temp['due_date'].apply(lambda x: 0 if x > '2023-01-03' else 1)

In [136]:
temp.sample(5)

,business_id,due_date,pilot_flag
1788,e6d11222-f9c7-470e-ae06-3f2de676e892,2022-09-10,1
986,818395b0-61da-4256-8d0a-fd6546ef70a8,2022-09-02,1
1881,f4d7772a-7edd-457f-8bef-98d9791f9fd8,2023-02-13,0
443,3754cacc-128b-4bda-8b3d-d27a76560bbf,2023-02-25,0
1449,bccfa5c0-6c15-498d-94ab-f516d43c90bc,2022-08-05,1


In [137]:
temp['pilot_flag'].value_counts()

0    1312
1     661
Name: pilot_flag, dtype: int64

In [138]:
dff = dff.merge(temp[['business_id','pilot_flag']],how='left', left_on = 'business_id', right_on = 'business_id')

In [139]:
dff.columns

Index(['business_id', 'due_date', 'DPD_plus_15_next', 'DPD_plus_3_last_2',
       'DPD_plus_15_prev', 'emi_no', 'DPD_plus_3', 'count_pd_deposit_1m',
       'count_ach_debit_1m', 'count_ach_credit_1m',
       ...
       'amount_cashin_2m_prev', 'distinct_draws_1m_prev',
       'distinct_draws_2m_prev', 'distinct_cashins_1m_prev',
       'distinct_cashins_2m_prev', 'ratio_amt_drawn_mom',
       'ratio_amt_cashin_mom', 'ratio_count_drawn_mom',
       'ratio_count_cashin_mom', 'pilot_flag'],
      dtype='object', length=121)

In [49]:
dff.shape

(4925, 121)

In [140]:
dff.groupby('pilot_flag')['DPD_plus_15_next'].sum()

pilot_flag
0     25.0
1    280.0
Name: DPD_plus_15_next, dtype: float64

In [141]:
dff['pilot_period_flag'] = dff['due_date'].apply(lambda x: 1 if x < '2022-12-20' else 0)

In [142]:
dff.groupby('pilot_period_flag')['DPD_plus_15_next'].sum()

pilot_period_flag
0     99.0
1    206.0
Name: DPD_plus_15_next, dtype: float64

In [66]:
dff['amount_debited_2m'].mean()

29076.153370558375

In [67]:
dff['amount_debited_2m'].describe()

count      4925.000000
mean      29076.153371
std       50996.016956
min           0.000000
25%        5458.670000
50%       12753.910000
75%       30141.830000
max      474715.400000
Name: amount_debited_2m, dtype: float64

In [68]:
dff['amount_credited_2m'].describe()

count      4925.000000
mean      28882.961304
std       54212.226953
min           0.000000
25%        5328.980000
50%       12650.720000
75%       29133.650000
max      517482.900000
Name: amount_credited_2m, dtype: float64

In [143]:
dff.groupby('emi_no')['DPD_plus_15_next'].sum()

emi_no
1     95.0
2    104.0
3     34.0
4     54.0
5     18.0
6      0.0
Name: DPD_plus_15_next, dtype: float64

In [144]:
import plotly.express as px
fig = px.bar(dff.groupby('emi_no')['DPD_plus_3','DPD_plus_15_next'].sum(),text_auto=True,width=800, height=400,barmode = 'group')
fig.update_traces(width=0.5)
fig.show()

/var/folders/v3/dy2p0xqd3mz5bvjtls49l7t80000gn/T/ipykernel_28973/2697453092.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [145]:
import plotly.express as px
fig = px.bar(dff.groupby(['pilot_flag'])['DPD_plus_15_next'].sum(),text_auto=True,width=800, height=400)
fig.update_traces(width=0.5)
fig.show()

In [146]:
import plotly.express as px
fig = px.bar(dff.groupby(['DPD_plus_3'])['DPD_plus_15_next'].sum(),text_auto=True,width=800, height=400)
fig.update_traces(width=0.4)
fig.show()

In [160]:
ddf = querySno(r"""
select json_extract_path_text(predict_meta, 'business_id') as business_id, json_extract_path_text(predict_meta, 'default_bin') as rs2_bin , novo_risk_score , fico_score
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS a 
left join  "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."CUSTOMER_CREDIT_PROFILES" b 
on a.lending_business_id = b.lending_business_id
""")



In [162]:
ldf = querySno(r""" 
select business_id, rs2_bin, rs2_score from PROD_DB.ADHOC.LENDING_PRESCREEN_2_DEC_26TH
where business_id in {}
""".format(tuple(dff.loc[dff['rs2_bin'].isnull(),'business_id']))
)

In [148]:
ddf.head()

,business_id,rs2_bin,novo_risk_score,fico_score
0,0a057d72-be21-4693-a225-9a9e8a683f23,4,880.0,695
1,bf8ec046-4e61-4acc-bdf1-992c752d50d3,4,663.0,681
2,a859e36b-2352-46f3-b7d3-ccb3bcfba2e1,3,477.0,630
3,b14635e3-50b9-489f-8971-70f0968369b0,1,249.0,563
4,f934cd75-6f22-459a-91b5-01d1bf0f133d,3,565.0,710


In [164]:
ldf.head()

,business_id,rs2_bin,rs2_score
0,679b7377-3974-4f0e-ac0f-4e0689417869,1,172.861789
1,acaaf2ad-b1d6-4164-90d1-5a968b89d2c8,1,21.642299
2,aaca717a-bb9d-4525-b5b5-dde64d021e91,5,748.948392
3,cae8cd40-3dc7-4754-bf58-331c0883c233,5,743.454921
4,28a85bc0-c5b7-461a-ae97-6336d9d2b814,1,17.929687


In [166]:
print(ddf.shape)
print(ldf.shape)

(7307, 4)
(245, 3)


In [165]:
ddf = ddf.drop_duplicates(subset=['business_id'])
ldf = ldf.drop_duplicates(subset=['business_id'])

In [151]:
dff = dff.merge(ddf, how='left', left_on='business_id', right_on = 'business_id')
temp = dff.merge(ldf, how='left', left_on='business_id', right_on = 'business_id')

In [186]:
temp[['rs2_bin_x','rs2_bin_y','novo_risk_score','rs2_score']] = temp[['rs2_bin_x','rs2_bin_y','novo_risk_score','rs2_score']].astype(float).fillna(0)

In [187]:
temp['rs2_bin'] = temp['rs2_bin_x'] + temp['rs2_bin_y']
temp['rs2_score_c'] = temp['novo_risk_score'] + temp['rs2_score']
temp.drop(['rs2_bin_x','novo_risk_score','rs2_bin_y','rs2_score'],axis=1,inplace=True)
dff = temp.rename({'rs2_score_c': 'rs2_score'}, axis=1) 

In [193]:
dff.head()

,business_id,due_date,DPD_plus_15_next,DPD_plus_3_last_2,DPD_plus_15_prev,emi_no,DPD_plus_3,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,count_card_withdrawal_1m,count_mrdc_credit_1m,count_pd_deposit_2m,count_ach_debit_2m,count_ach_credit_2m,count_card_withdrawal_2m,count_mrdc_credit_2m,amount_pd_deposit_1m,amount_ach_debit_1m,amount_ach_credit_1m,amount_card_withdrawal_1m,amount_mrdc_credit_1m,amount_pd_deposit_2m,amount_ach_debit_2m,amount_ach_credit_2m,amount_card_withdrawal_2m,amount_mrdc_credit_2m,pd_deposit_size_1m,ach_debit_size_1m,ach_credit_size_1m,card_withdrawal_size_1m,mrdc_credit_size_1m,pd_deposit_size_2m,ach_debit_size_2m,ach_credit_size_2m,card_withdrawal_size_2m,mrdc_credit_size_2m,amount_credited_1m,distinct_credit_txns_1m,credit_size_1m,median_amount_credited_1m,stddev_amount_credited_1m,amount_credited_2m,distinct_credit_txns_2m,credit_size_2m,median_amount_credited_2m,stddev_amount_credited_2m,amount_debited_1m,distinct_debit_txns_1m,debit_size_1m,median_amount_debited_1m,stddev_amount_debited_1m,amount_debited_2m,distinct_debit_txns_2m,debit_size_2m,median_amount_debited_2m,stddev_amount_debited_2m,ratio_debit_credit_1m,ratio_debit_credit_frequency_1m,ratio_debit_credit_2m,ratio_debit_credit_frequency_2m,ratio_pd_amt_total_credit_1m,ratio_ach_debit_total_debit_1m,ratio_ach_credit_total_credit_1m,ratio_card_withdrawal_total_debit_1m,ratio_mrdc_credit_total_credit_1m,ratio_pd_amt_total_credit_2m,ratio_ach_debit_total_debit_2m,ratio_ach_credit_total_credit_2m,ratio_card_withdrawal_total_debit_2m,ratio_mrdc_credit_total_credit_2m,ratio_pd_freq_total_credit_1m,ratio_ach_debit_freq_total_debit_1m,ratio_ach_credit_freq_total_credit_1m,ratio_card_withdrawal_freq_total_debit_1m,ratio_mrdc_credit_freq_total_credit_1m,ratio_pd_freq_total_credit_2m,ratio_ach_debit_freq_total_debit_2m,ratio_ach_credit_freq_total_credit_2m,ratio_card_withdrawal_freq_total_debit_2m,ratio_mrdc_credit_freq_total_credit_2m,ratio_ach_debit_credit_1m,ratio_ach_debit_credit_2m,ratio_ach_debit_credit_freq_1m,ratio_ach_debit_credit_freq_2m,ach_d_size_1m,ach_d_size_2m,ach_c_size_1m,ach_c_size_2m,od_count_1m,zero_balance_count_1m,avg_running_balance_1m,median_running_balance_1m,stddev_running_balance_1m,od_count_2m,zero_balance_count_2m,avg_running_balance_2m,median_running_balance_2m,stddev_running_balance_2m,avg_running_balance_2nd_m,median_running_balance_2nd_m,stddev_running_balance_2nd_m,ratio_avg_running_balance_1m_2m,revenue_past_1m,revenue_past_2m,total_revenue_2m,ratio_revenue_1m_2m,credit_limit,amount_drawn_1m_prev,amount_drawn_2m_prev,amount_cashin_1m_prev,amount_cashin_2m_prev,distinct_draws_1m_prev,distinct_draws_2m_prev,distinct_cashins_1m_prev,distinct_cashins_2m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom,pilot_flag,pilot_period_flag,fico_score,rs2_bin,rs2_score
0,c616b39b-aef7-4156-973a-08278aa74536,2023-04-03,0.0,0.0,0.0,1,0,0.0,1.0,0.0,35.0,0.0,0.0,1.0,0.0,38.0,0.0,0.00,295.36,0.00,3962.06,0.0,0.00,295.36,0.00,4102.83,0.0,0.000000,147.680000,0.000000,110.057222,0.0,0.000000,147.680000,0.00000,105.200769,0.0,1800.00,2.0,600.000000,0.00,195.239512,2800.00,3.0,700.000000,0.00,244.616840,4257.42,36.0,115.065405,35.885,192.561543,4398.19,39.0,109.954750,42.995,191.137528,2.363920,12.000000,1.570221,9.750000,0.000000,0.069375,0.000000,0.930625,0.0,0.000000,0.067155,0.000000,0.932845,0.0,0.000000,0.027027,0.000000,0.945946,0.0,0.000000,0.025000,0.000000,0.950000,0.0,295.360000,295.360000,1.000000,1.000000,147.680000,147.680000,0.000000,0.00000,0,0,17737.240094,18065.85,1433.419027,0,0,17883.244097,18065.85,1689.488039,19435.753333,21236.060,2998.968352,0.912562,12494.28,0.00,12494.28,12494.280000,11600.0,1000.0,2800.0,2884.00,0.00,1.0,3.0,2.0,0.0,0.357015,2884.000000,0.25,2.000000,0,0,749,5.0,971.0
1,cc861415-fd34-4654-8b97-dab0794938dc,2023-04-03,0.0,0.0,0.0,1,0,0.0,14.0,4.0,31.0,0.0,0.0,15.0,4.0,31.0,0.0,0.00,10562.34,15750.00,1948.69,0.0,0.00,10902.18,15750.00,1948.69,0.0

In [194]:
import plotly.express as px
fig = px.bar(dff.groupby(['rs2_bin'])['DPD_plus_15_next'].sum(),text_auto=True,width=800, height=400)
fig.update_traces(width=0.4)
fig.show()

In [153]:
pd.crosstab(dff['rs2_bin'],dff['emi_no'])

emi_no,1,2,3,4,5,6
rs2_bin,,,,,,
1,73,71,71,50,49,42
2,113,111,106,95,91,79
3,433,315,158,116,110,99
4,414,243,68,47,43,37
5,696,397,36,23,22,13


In [195]:
pd.pivot_table(dff[['rs2_bin','emi_no','DPD_plus_15_next']],index = 'rs2_bin',
               columns = 'emi_no',aggfunc ='sum').to_clipboard()

In [159]:
# tuple(dff.loc[dff['rs2_bin'].isnull(),'business_id'])

In [196]:
fig = px.imshow(pd.pivot_table(dff[['rs2_bin','emi_no','DPD_plus_15_next']],index = 'rs2_bin',
               columns = 'emi_no',aggfunc ='sum'), text_auto=True, width=800, height=400)
fig.show()

In [ ]:
dff.groupby(['rs2_bin','emi_no'])['DPD_plus_15_next'].sum()

In [50]:
# # All cycles
# print('count of DPD_plus_15 - ',dff['DPD_plus_15'].sum())
# print('count of all cycles - ',len(dff))
# print('DPD_plus_15 % -',dff['DPD_plus_15'].sum()/len(dff))

In [51]:
# # At Business_id level
# biz_df = dff.groupby(['business_id'], as_index=False).agg(DPD_plus_3_count = ('DPD_plus_3','sum'), DPD_plus_15_count = ('DPD_plus_15', 'sum'), count_of_cycles = ('DPD_plus_15','count')).reset_index()
# biz_df['DPD_plus_3_%'] = (biz_df['DPD_plus_3_count']/biz_df['count_of_cycles'])*100
# biz_df['DPD_plus_15_%'] = (biz_df['DPD_plus_15_count']/biz_df['count_of_cycles'])*100
# biz_df.sort_values(by='DPD_plus_15_%',ascending=False).reset_index()

In [52]:
# print(biz_df[biz_df['DPD_plus_3_count']>0].shape)
# print(biz_df[biz_df['DPD_plus_15_count']>0].shape)
# print(biz_df.shape)
# # print(biz_df[biz_df['DPD_plus_15_count']>0])

In [53]:
# # Excluding 1st & 2nd cycle
# exc = dff[~(dff['emi_no'].isin([1,2]))]
# print('count of DPD_plus_15 - ',exc['DPD_plus_15'].sum())
# print('count of all cycles - ',len(exc))
# print('DPD_plus_15 % -',exc['DPD_plus_15'].sum()/len(exc))

In [54]:
# temp = dff[~(dff['emi_no'].isin([1,6]))]
# print(temp.shape)
# print(temp['DPD_plus_15_next'].sum())

In [55]:
# At Business_id level


In [197]:
# dff.to_pickle("/Users/shashankgupta/Documents/code/git_project/re_decisioning/Data/dataset_2.pkl")  